# Pursuit-Evasion Game Demo

In this demo, we will:
1. Implement a linear-programming solver for static matrix games, which will serve as the building block for our stochastic game solver;
2. Initialize a pursuit–evasion game (PEG) instance as an example of a stochastic game;
3. Implement the code that generates the Q-matrix at each state;
4. Implement the value update step;
5. Combine Q-matrix generation and value updates into a concise `NashSolver` class;
6. Solve the PEG and visualize the results.

In [1]:
import numpy as np
from typing import List, Tuple
from scipy.optimize import linprog
import sys, os
sys.path.append(os.path.dirname(os.path.abspath(".")))

### Step 1: Linear program for a static matrix game

We start with a static zero-sum matrix game with payoff matrix $A \in \mathbb{R}^{m \times n}$.  
The **row player** chooses a mixed strategy $x \in \Delta_m$ and wants to maximize the expected payoff, while the **column player** chooses a mixed strategy $y \in \Delta_n$ and wants to minimize it.

The value of the game is
\begin{equation}
v^\star \;=\; \max_{x \in \Delta_m} \; \min_{y \in \Delta_n} \; x^\top A y.
\end{equation}

We can rewrite the max–min problem as the following **linear program** in the variables $(x, v)$ for the **row player**:

\begin{aligned}
\max_{x, v} \quad & v \\
\text{s.t.} \quad & A^\top x \;\ge\; v \mathbf{1}_n, \\
                  & \mathbf{1}_m^\top x = 1, \\
                  & x \ge 0.
\end{aligned}

Here, $v$ is the game value (the guaranteed payoff for the row player), and $p$ is the optimal mixed strategy over rows.

### Solving linear programs using `scipy.optimize.linprog`

`scipy.optimize.linprog` solves linear programs in the following standard form:

\begin{aligned}
\min_x \quad & c^\top x \\
\text{s.t.} \quad 
& A_{\text{ub}} x \le b_{\text{ub}}, \\
& A_{\text{eq}} x = b_{\text{eq}}, \\
& \ell \le x \le u,
\end{aligned}

where:
- $c \in \mathbb{R}^n$ is the cost vector,
- $A_{\text{ub}} \in \mathbb{R}^{m \times n}$, $b_{\text{ub}} \in \mathbb{R}^m$ define inequality constraints,
- $A_{\text{eq}} \in \mathbb{R}^{k \times n}$, $b_{\text{eq}} \in \mathbb{R}^k$ define equality constraints,
- $\ell, u$ define lower/upper bounds on each variable.

The Python interface is:

```python
from scipy.optimize import linprog

result = linprog(
    c,
    A_ub=A_ub, b_ub=b_ub,
    A_eq=A_eq, b_eq=b_eq,
    bounds=bounds,
    method="highs" 
)
```

The solution is stored in the returned object:
```python
result.x      # optimal decision variables
result.fun    # optimal objective value
```

Now, we can implement the static matrix game solver.

In [2]:
def linprog_solve(payoff_matrix: np.ndarray) -> (float, np.ndarray, np.ndarray):
    '''
    :param payoff_matrix: M X N Matrix. The entry of the jointly selected row and column represents the winnings of the
                          row player and the loss of the column player
                          i.e. row maximizes and column minimizes
    :return: the value of the game, the optimal policy for the row player and the optimal policy for the column player
    '''
    m, n = payoff_matrix.shape[0], payoff_matrix.shape[1]

    
    # Setting up the inputs to the scipy.optimize.linprog
    # Decision variables x1, x2, ..., xn, v
    # Objective is to maximize v, thus objective vector c is 0*y1+0*y2+...+0*yn - v
    
    C = []
    for i in range(n):
        C.append(0)
    C.append(-1)

    
    # Optimality constraints: value_matrix[i_row, :] @ x >= v for all i_row
    A_ub = []
    for i_row in range(m):
        col = payoff_matrix[i_row, :]
        constraint_row = []
        for item in col:
            constraint_row.append(-item)
        constraint_row.append(1)
        A_ub.append(constraint_row)
    B_ub = []
    for i in range(m):
        B_ub.append(0)

    # Probability constraint: x1 + x2 + ... + xn = 1
    A_eq = []
    A_eq_row = []
    for i in range(n):
        A_eq_row.append(1)
    A_eq_row.append(0)
    A_eq.append(A_eq_row)
    B_eq = [1]

    bounds = []
    for i in range(n):
        bounds.append((0, 1))
    bounds.append((None, None))

    # Linprog solves the problem:
    res = linprog(C, A_ub=A_ub, b_ub=B_ub, A_eq=A_eq, b_eq=B_eq, bounds=bounds, method='highs')

    # Col player's policy is retrieved from the x variable
    policy_col = res['x'][:-1]
    for i, p in enumerate(policy_col):
        if p < 0:
            policy_col[i] = 0
    policy_col /= sum(policy_col)

    # Row player's policy is retrieved from the dual variable
    policy_row = -res.ineqlin.marginals

    return res['fun'], policy_row, policy_col

Let's now quickly test the solver we just implemented. Let's test it on a Rock Paper Scissor problem!

In [3]:
rps_payoff = np.array(
    [[0, 1, -1],
     [-1, 0, 1],
     [1, -1, 0]
    ]
)

rps_value, row_policy, col_policy = linprog_solve(rps_payoff)
print("game value: {}".format(rps_value))
print("row player policy: {}".format(row_policy))
print("col player policy: {}".format(col_policy))

game value: 0.0
row player policy: [0.33333333 0.33333333 0.33333333]
col player policy: [0.33333333 0.33333333 0.33333333]


### Step 2: Initialize a pursuit-evasion game
- Pursuer maximizes, evader minimizes.
- A reward of +1 is assigned when capture happens, otherwise reward is zero. 
- Occupancy_matrix defines the grid environment with 1 representing obstacles and 0 for free space.
- Capture radius: If the distance between the pursuer and the evader is leq capture radius then the evader is captured.
- Action spaces are the movement vectors. We give the pursuer an advantage by allowing it to move diagonally, so that a capture can be ensured.
- transition_eps denotes for the probability of agent staying at its current position, even when it intends to move. Use for stochastic transitions.

The PEG code is already implemented. 
We can now initialize an instance of a PEG with the following `occupancy_matrix`, where 0 is free space and 1 is obstacle.
While you can change the environment freely, keep in mind that for a large grid size, it may take a while to generate the game due to the construction of the transition and reward matrices.


In [5]:
from game_example.peg import PursuitEvasionGame
occupancy_matrix = np.array([
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 1, 1, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 1, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
])

peg = PursuitEvasionGame(occupancy_matrix=occupancy_matrix, capture_radius=1, transition_eps=0.2, gamma=0.9)

Initializing Pursuit-Evasion Game...
    Generating rewards for PEG...
    Generating compressed transitions for PEG...
PEG initialized!


### Solve the game using the Nash Solver

Now you can solve the PEG with the Nash solver. Some of the key options for the solver are listed as follows:
- verbose: whether or not to print the progress of the solver. 
- eps: The threshold for value iteration. Once the 2-norm between the old value vector and the new one is below this threshold, the value is considered converged.
- n_policy_eval: The number of policy evaluation steps performed before calling the linear program (LP) solver to compute the Nash value from the Q-function. Empirically, having a few policy evaluations before computing the Nash value helps reduce the number of calling the LP solvers, which saves significant amount of computation time.
- n_workers: number of workers for parallelizing the computation of Nash values (LP).

In [6]:
from game_example.solver import NashSolver
solver = NashSolver(game=peg)
solver.solve(verbose=True, n_policy_eval=5, eps=0.1, n_workers=4)

Solving Nash equilibrium of a game with 2209 states
Using 4 workers for parallel computation
Iter       Total Time Difference      VI Time    LP Time    Policy Eval Time
0          0.5082     15.5242         0.0014     0.4531     0.0123    
1          0.9068     17.9747         0.0014     0.3705     0.0264    
2          1.307      14.2617         0.0014     0.3735     0.0251    
3          1.6959     11.0432         0.0014     0.3622     0.025     
4          2.0914     6.2564          0.0014     0.3691     0.0248    
5          2.4875     1.5634          0.0014     0.3694     0.025     
6          2.8766     0.6481          0.0014     0.3626     0.0248    
7          3.2707     0.3121          0.0014     0.3677     0.0247    
8          3.6691     0.1659          0.0014     0.3718     0.025     
9          4.0632     0.0881          0.0014     0.3676     0.025     
Value iterations converged!
Matrix Game solver called 22090 times


### Visualization 
To visualize the obtained policies, we first generate the state/position trajectories for both agents. Note that the Nash equilibirum policies are stochastic, so you may end up with a different trajectory every time you run the following trajectory generation code. 

In [8]:
# Gather the policy from the solver
policy_p, policy_e = solver.policy_1, solver.policy_2

# Simulate the trajectory
pos_p, pos_e = (0,0), (7, 7)                               # initial positions
state = peg.pos2game_state(pos_1=pos_p, pos_2=pos_e)         # initial game state

state_trj = [state]
pursuer_pos_trj, evader_pos_trj = [pos_p], [pos_e]

while not peg.is_terminal(state):
    # get action distributions
    action_dist_p, action_dist_e = policy_p[state], policy_e[state]

    # selection actions
    action_space_p, action_space_e = peg.action_mappings_1[state], peg.action_mappings_2[state]
    action_index_p = np.random.choice(a=list(range(len(action_space_p))), p=action_dist_p)
    action_index_e = np.random.choice(a=list(range(len(action_space_e))), p=action_dist_e)
    action_p, action_e = action_space_p[action_index_p], action_space_e[action_index_e]

    # update positions and state
    pos_p = (pos_p[0] + action_p[0], pos_p[1] + action_p[1])
    pos_e = (pos_e[0] + action_e[0], pos_e[1] + action_e[1])
    state = peg.pos2game_state(pos_p, pos_e)

    # store the trajectory
    state_trj.append(state)
    pursuer_pos_trj.append(pos_p)
    evader_pos_trj.append(pos_e)

print("Simulation completed. Total steps: ", len(state_trj))

Simulation completed. Total steps:  19


Now we can animate the trajectories. You may need to update the matplotlib to ensure that the inline animation works properly.
Enjoy!

In [9]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as lines
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation

%matplotlib inline

# initialize plot
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff()
fig, ax = plt.subplots(figsize=(4,4))

n_rows, n_cols = occupancy_matrix.shape
def animate(t):
    plt.cla()
    ax.axis([-0.1, n_rows+0.1, -0.1, n_cols +0.1])
    ax.axis("off")
    ax.set_aspect('equal', adjustable='box')

    # render obstacles and grids
    for i in range(n_rows):
        for j in range(n_cols):
            if occupancy_matrix[i, j] == 1:
                ax.add_patch(patches.Rectangle((j, i), 1.0, 1.0, facecolor="gray"))
    for i in range(n_rows + 1):
        ax.add_artist(lines.Line2D([0, n_cols], [i, i], color="k", linestyle=":", linewidth=0.5))
    for j in range(n_cols + 1):
        ax.add_artist(lines.Line2D([j, j], [n_rows, 0], color="k", linestyle=":", linewidth=0.5))

    # render pursuer
    pos_p = pursuer_pos_trj[t]
    ax.add_artist(patches.Circle((pos_p[1] + 0.5, pos_p[0] + 0.5), radius=0.2, color='r'))

    # render capture radius
    row, col = pos_p[0] + 0.5, pos_p[1] + 0.5
    radius = peg.capture_radius + 0.5
    row_start, row_end = row - radius, row + radius
    col_start, col_end = col - radius, col + radius
    # row lines
    ax.add_artist(lines.Line2D([col_start, col_end], [row_start, row_start], color='r', linestyle="--",
                                       linewidth=1))
    ax.add_artist(lines.Line2D([col_start, col_end], [row_end, row_end], color='r', linestyle="--",
                                       linewidth=1))
    # col lines
    ax.add_artist(lines.Line2D([col_start, col_start], [row_start, row_end], color='r', linestyle="--",
                                       linewidth=1))
    ax.add_artist(lines.Line2D([col_end, col_end], [row_start, row_end], color='r', linestyle="--",
                                       linewidth=1))

    # render evader
    pos_e = evader_pos_trj[t]
    ax.add_artist(patches.Circle((pos_e[1] + 0.5, pos_e[0] + 0.5), radius=0.2, color='b'))


FuncAnimation(fig, animate, frames=len(state_trj), interval=100)